In [1]:
import numpy as np
import tensorflow as tf
import glob
import PIL
from PIL import Image  
from skimage import color
import shutil
import json
import os
from skimage.feature import hog

In [10]:
def preprocessing_image(img_path,image_size):
    original = Image.open(img_path)  
    resized_image = original.resize(image_size)
    resized_image.load()
    data = np.asarray( resized_image, dtype="int32" )
    grayscale_image = color.rgb2gray(data)
    features = hog(grayscale_image, 
                   orientations=30, 
                   pixels_per_cell=(16, 16), 
                   cells_per_block=(2, 2), 
                   block_norm='L2-Hys', 
                   visualize=False, 
                   transform_sqrt=False, 
                   feature_vector=True, 
                   multichannel=None)
    return features

In [3]:
def data_generation(img_dir,resize_image):
    image_data = []
    img_label = []
    label_counter=0
    for each_dir in sorted(glob.glob(img_dir+"\\"+"*"+"\\")):
        for each in sorted(glob.glob(each_dir+"*.JPG")):
            preprocessed_image = preprocessing_image(each,resize_image)
            image_data.append(preprocessed_image)
            img_label.append(int(os.path.basename(os.path.dirname(each))))
        for each in sorted(glob.glob(each_dir+"*.PNG")):
            preprocessed_image = preprocessing_image(each,resize_image)
            image_data.append(preprocessed_image)
            img_label.append(int(os.path.basename(os.path.dirname(each))))
    return np.array(image_data), np.array(img_label)

In [4]:
def train_test_divider(dir_label, img_dir, train_obj_dir, test_obj_dir, train_dir, test_dir):
    train_data = [os.path.basename(i).split(".")[0] for i in glob.glob(train_obj_dir+"*.JPG")]
    test_data = [os.path.basename(i).split(".")[0] for i in glob.glob(test_obj_dir+"*.JPG")]
    for each in glob.glob(img_dir+"*.PNG"):
        file_basename = os.path.basename(each).split(".")[0]
        if file_basename in train_data:
            try:
                os.mkdir(train_dir + "\\" + dir_label) 
            except FileExistsError:
                pass
            shutil.copy2(each,train_dir + "\\" + dir_label)
        elif file_basename in test_data:
            try:
                os.mkdir(test_dir + "\\" + dir_label) 
            except FileExistsError:
                pass
            shutil.copy2(each,test_dir + "\\" + dir_label)
    print("Train Test Split Complete")

In [5]:
def copy_files(src_dir, target_dir):
    src_dir = glob.glob(src_dir+"*"+"\\")
    for i in src_dir:
        for each in glob.glob(i+"*.JPG"):
            shutil.copy2(each,target_dir + "\\"+ os.path.basename(os.path.dirname(each)))
    print("Copying Files complete")

In [11]:
def data_preparation_pipeline():
    with open('config.json') as json_file:
        data = json.load(json_file)
    all_label_dir = glob.glob(data["object_classifier_directory"]["brand_img_dir"]+"\\"+"*"+"\\")
    try:
        os.mkdir(data["object_classifier_directory"]["train_brand_dir"])
        os.mkdir(data["object_classifier_directory"]["test_brand_dir"])
    except FileExistsError:
        pass
    for each in all_label_dir:
        train_test_divider(os.path.basename(os.path.dirname(each)), 
                           each,
                           data["object_detector_directory"]["train_object_dir"],
                           data["object_detector_directory"]["test_object_dir"],
                           data["object_classifier_directory"]["train_brand_dir"],
                           data["object_classifier_directory"]["test_brand_dir"],
                           )
    copy_files(data["object_classifier_directory"]["brand_dir"], data["object_classifier_directory"]["train_brand_dir"])
    full_product_data, full_product_label = data_generation(data["object_classifier_directory"]["train_brand_dir"],(128,128))
    test_product_data, test_product_label = data_generation(data["object_classifier_directory"]["test_brand_dir"], (128,128))
    os.mkdir("features")
    os.mkdir("scalers")
    os.mkdir("models")
    np.save("features\\hog_brand_train_array.npy",full_product_data)
    np.save("features\\hog_brand_test_array.npy",test_product_data)
    np.save("features\\hog_brand_train_label", full_product_label)
    np.save("features\\hog_brand_test_label", test_product_label)
    print("Final Feature Arrays for Classification Created")

In [12]:
data_preparation_pipeline()

Final Feature Arrays for Classification Created
